# Compute Transmission × FHSZ Exposure (Length-Weighted)

This notebook computes screening-level wildfire hazard exposure for publicly identifiable ≥115 kV transmission lines by intersecting line geometry with CAL FIRE Fire Hazard Severity Zone (FHSZ) polygons. All spatial computation is performed in PostGIS and cached for reuse. Length is measured in EPSG:3310 (California Albers) and reported in kilometers.

## Inputs (PostGIS)
- `transmission_lines` (canonical lines; stored in EPSG:4326)
- `hazard_fhsz` (canonical hazard zones; stored in EPSG:4326)

## Outputs (PostGIS)
- `tx_lines_3310` (lines transformed to EPSG:3310)
- `hazard_fhsz_3310` (hazard polygons transformed to EPSG:3310)
- `tx_overlap_fhsz_by_line` (line_id, hazard_class, overlap_km)

## Outputs (Files)
- `outputs/tables/tx_length_by_fhsz_class.csv`
- `outputs/tables/tx_overlap_fhsz_by_line.csv`

### Imports

In [1]:
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

### Define Paths

In [2]:
CWD = Path.cwd()
ROOT = CWD.parent if CWD.name.lower() == "notebooks" else CWD

OUTPUT_TABLES = ROOT / "outputs" / "tables"
OUTPUT_TABLES.mkdir(parents=True, exist_ok=True)

load_dotenv(ROOT / ".env")
print("ROOT:", ROOT)
print("OUTPUT_TABLES:", OUTPUT_TABLES)

ROOT: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure
OUTPUT_TABLES: C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables


## Database Connection

In [3]:
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "wildfire_grid")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

if not DB_USER or not DB_PASSWORD:
    raise ValueError("Missing DB_USER or DB_PASSWORD in environment (.env).")

db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)

with engine.begin() as conn:
    conn.execute(text("SELECT 1;"))

print("DB:", DB_NAME, "@", DB_HOST, DB_PORT)

DB: wildfire_grid @ localhost 5432


## Pre-Checks 

In [4]:
check_sql = """
SELECT
  (SELECT COUNT(*) FROM transmission_lines) AS n_tx,
  (SELECT COUNT(*) FROM hazard_fhsz) AS n_fhsz;
"""
pd.read_sql(check_sql, engine)

,n_tx,n_fhsz
0,2167,28175


## Create EPSG:3310 Cached Geometry Tables

In [5]:
sql_cache_3310 = """
DROP TABLE IF EXISTS tx_lines_3310;
CREATE TABLE tx_lines_3310 AS
SELECT
  line_id,
  voltage_kv,
  owner,
  ST_Transform(ST_MakeValid(geom), 3310) AS geom
FROM transmission_lines;

ALTER TABLE tx_lines_3310 ADD PRIMARY KEY (line_id);
CREATE INDEX IF NOT EXISTS idx_tx_lines_3310_geom_gist ON tx_lines_3310 USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_tx_lines_3310_voltage ON tx_lines_3310 (voltage_kv);
CREATE INDEX IF NOT EXISTS idx_tx_lines_3310_owner ON tx_lines_3310 (owner);
ANALYZE tx_lines_3310;

DROP TABLE IF EXISTS hazard_fhsz_3310;
CREATE TABLE hazard_fhsz_3310 AS
SELECT
  hazard_class,
  ST_Transform(ST_MakeValid(geom), 3310) AS geom
FROM hazard_fhsz
WHERE hazard_class IS NOT NULL;

CREATE INDEX IF NOT EXISTS idx_hazard_fhsz_3310_geom_gist ON hazard_fhsz_3310 USING GIST (geom);
CREATE INDEX IF NOT EXISTS idx_hazard_fhsz_3310_class ON hazard_fhsz_3310 (hazard_class);
ANALYZE hazard_fhsz_3310;
"""

with engine.begin() as conn:
    conn.execute(text(sql_cache_3310))

print("Created: tx_lines_3310, hazard_fhsz_3310")

Created: tx_lines_3310, hazard_fhsz_3310


## Compute Overlap

In [6]:
sql_overlap = """
DROP TABLE IF EXISTS tx_overlap_fhsz_by_line;

CREATE TABLE tx_overlap_fhsz_by_line AS
WITH candidates AS (
  SELECT
    t.line_id,
    h.hazard_class,
    ST_Intersection(t.geom, h.geom) AS inter_geom
  FROM tx_lines_3310 t
  JOIN hazard_fhsz_3310 h
    ON ST_Intersects(t.geom, h.geom)
),
lines_only AS (
  SELECT
    line_id,
    hazard_class,
    ST_CollectionExtract(inter_geom, 2) AS inter_line
  FROM candidates
)
SELECT
  line_id,
  hazard_class,
  (ST_Length(inter_line) / 1000.0) AS overlap_km
FROM lines_only
WHERE inter_line IS NOT NULL
  AND NOT ST_IsEmpty(inter_line);

CREATE INDEX IF NOT EXISTS idx_tx_overlap_fhsz_line ON tx_overlap_fhsz_by_line (line_id);
CREATE INDEX IF NOT EXISTS idx_tx_overlap_fhsz_class ON tx_overlap_fhsz_by_line (hazard_class);
ANALYZE tx_overlap_fhsz_by_line;
"""

with engine.begin() as conn:
    conn.execute(text(sql_overlap))

print("Created: tx_overlap_fhsz_by_line")

Created: tx_overlap_fhsz_by_line


### Total Lengths by Hazard Class

In [7]:
summary_sql = """
SELECT
  hazard_class,
  SUM(overlap_km) AS overlap_km
FROM tx_overlap_fhsz_by_line
GROUP BY 1
ORDER BY overlap_km DESC;
"""
df_class = pd.read_sql(summary_sql, engine)
df_class

,hazard_class,overlap_km
0,NonWildland,13908.787465
1,Very High,7126.069250
2,High,6876.888507
3,Moderate,6286.233349


## Export CSVs

In [8]:
df_class.to_csv(OUTPUT_TABLES / "tx_length_by_fhsz_class.csv", index=False)

df_by_line = pd.read_sql(
    """
    SELECT line_id, hazard_class, overlap_km
    FROM tx_overlap_fhsz_by_line
    ORDER BY overlap_km DESC;
    """,
    engine
)
df_by_line.to_csv(OUTPUT_TABLES / "tx_overlap_fhsz_by_line.csv", index=False)

print("Wrote:")
print("-", OUTPUT_TABLES / "tx_length_by_fhsz_class.csv")
print("-", OUTPUT_TABLES / "tx_overlap_fhsz_by_line.csv")

Wrote:
- C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables\tx_length_by_fhsz_class.csv
- C:\dev\wildfire\Wildfire-Exposure-of-California-Transmission-Infrastructure\outputs\tables\tx_overlap_fhsz_by_line.csv
